In [14]:
import tensorflow as tf
from tensorflow.keras import layers

print(tf.VERSION)
print(tf.keras.__version__)

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

1.11.0
2.1.6-tf


## Normal model

In [15]:
print(x_train.shape)
print(x_test.shape)
print(y_train[0:3])
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=99)
print(x_train.shape)
print(x_valid.shape)
print(y_train[0:3])

(60000, 28, 28)
(10000, 28, 28)
[5 0 4]
(48000, 28, 28)
(12000, 28, 28)
[4 4 4]


In [18]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [19]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [21]:
%%time 
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
fit = model.fit(x_train, y_train, epochs=5, validation_data=(x_valid, y_valid))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 6s 123us/step - loss: 0.2419 - acc: 0.9283 - val_loss: 0.1243 - val_acc: 0.9638
Epoch 2/5
48000/48000 [==============================] - 7s 150us/step - loss: 0.1075 - acc: 0.9677 - val_loss: 0.0992 - val_acc: 0.9689
Epoch 3/5
48000/48000 [==============================] - 7s 154us/step - loss: 0.0746 - acc: 0.9772 - val_loss: 0.0818 - val_acc: 0.9764
Epoch 4/5
48000/48000 [==============================] - 9s 191us/step - loss: 0.0544 - acc: 0.9820 - val_loss: 0.0790 - val_acc: 0.9771
Epoch 5/5
48000/48000 [==============================] - 8s 164us/step - loss: 0.0472 - acc: 0.9841 - val_loss: 0.0733 - val_acc: 0.9778
Wall time: 37.8 s


In [30]:
%%time 
y_train_one_hot = tf.keras.utils.to_categorical(y_train, 10)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(x_train, y_train_one_hot, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 17s 277us/step - loss: 0.0207 - acc: 0.9934
Epoch 2/5
60000/60000 [==============================] - 20s 336us/step - loss: 0.0158 - acc: 0.9946
Epoch 3/5
60000/60000 [==============================] - 20s 327us/step - loss: 0.0163 - acc: 0.9948
Epoch 4/5
60000/60000 [==============================] - 18s 292us/step - loss: 0.0157 - acc: 0.9946
Epoch 5/5
60000/60000 [==============================] - 22s 370us/step - loss: 0.0159 - acc: 0.9947


In [56]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape)
print(x_test.shape)
model = tf.keras.models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28, 28,1)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [57]:
%%time 
y_train_one_hot = tf.keras.utils.to_categorical(y_train, 10)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(x_train, y_train_one_hot, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 194s 3ms/step - loss: 0.1162 - acc: 0.9637
Epoch 2/5
60000/60000 [==============================] - 186s 3ms/step - loss: 0.0388 - acc: 0.9881
Epoch 3/5
60000/60000 [==============================] - 187s 3ms/step - loss: 0.0239 - acc: 0.9925
Epoch 4/5
60000/60000 [==============================] - 180s 3ms/step - loss: 0.0149 - acc: 0.9948
Epoch 5/5
60000/60000 [==============================] - 184s 3ms/step - loss: 0.0104 - acc: 0.9967


# Define model
All layers must be defined in __init__() and run order in call()

<mark>TODO</mark>: what is compute_output_shape()

"[In case your layer modifies the shape of its input, you should specify here the shape transformation logic](https://keras.io/layers/writing-your-own-keras-layers/). This allows Keras to do automatic shape inference." (from keras page) - **mean, not necessary ?**

In [31]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.flatten_1= layers.Flatten(input_shape=(28, 28))
        self.dense_1 = layers.Dense(512, activation='relu')
        self.dropout_1 = layers.Dropout(0.2)
        self.dense_2 = layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.flatten_1(inputs)
        x = self.dense_1(x)
        x = self.dropout_1(x)
        return self.dense_2(x)
    
#     def compute_output_shape(self, input_shape):
#         shape = tf.TensorShape(input_shape).as_list()
#         shape[-1] = self.num_classes
#         return tf.TensorShape(shape)

In [32]:
model = MyModel(num_classes=10)
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(x_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
48000/48000 [==============================] - 7s 148us/step - loss: 0.3537 - acc: 0.8911
Epoch 2/5
48000/48000 [==============================] - 8s 159us/step - loss: 0.1191 - acc: 0.9655
Epoch 3/5
48000/48000 [==============================] - 8s 159us/step - loss: 0.0896 - acc: 0.9745
Epoch 4/5
48000/48000 [==============================] - 9s 183us/step - loss: 0.0754 - acc: 0.9791
Epoch 5/5
48000/48000 [==============================] - 9s 182us/step - loss: 0.0607 - acc: 0.9829


In [103]:
import numpy as np

class MyFlattenLayer(layers.Layer):
    def __init__(self, **kwargs):
#         (row, col) = output_shape
#         self.output_dim = row*col
#         self.input_shape = input_shape
        super(MyFlattenLayer, self).__init__(**kwargs)
        
    def bulid(self, input_shape):
        shape = tf.TensorShape((self.input_shape, 1))
        self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform',
                                  trainable=True)
        super(MyFlattenLayer, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
#         return inputs.reshape((, self.output_dim))
#         return inputs.reshape(inputs.shape[0], -1)
    
#     def comput_output_shape(self, input_shape):
#         shape = tf.TensorShape(input_shape).as_list()
#         shape[-1] = self.output_dim
#         return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return cls(**kwargs)
    
class MySoftmaxLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MySoftmaxLayer, self).__init__(**kwargs)
        
    def bulid(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform',
                                  trainable=True)
        super(MySoftmaxLayer, self).build(input_shape)
        
    def call(self, inputs):
        conv = tf.matmul(inputs, self.kernel)
        return tf.argmax(inputs, axis=-1)
    
    def comput_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return cls(**kwargs)

In [104]:
print(x_train.shape)
test = x_train.reshape((x_train.shape[0],-1))
print(test.shape)

(48000, 28, 28)
(48000, 784)


In [105]:
# model = tf.keras.Sequential()
# # model.add(MyFlattenLayer(10))
# # model.add(layers.Activation('softmax'))
# # model.add(layers.Dense(10, activation='softmax'))
# model.add(MySoftmaxLayer(10))
# model.add(layers.Flatten(input_shape=(28, 28)))

model = tf.keras.Sequential([
#     tf.keras.layers.Flatten(input_shape=(28, 28)),
    MyFlattenLayer(input_shape = (28,28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])



model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(x_train, y_train, batch_size=32, epochs=1)

AttributeError: 'MyFlattenLayer' object has no attribute 'kernel'